In [1]:
import numpy as np
from scipy import stats
import pandas as pd
import time
#import math

# Sklearn
import sklearn
# Preprocessor from sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Keras
import keras
# ANNs from keras
from keras.models import Sequential
from keras.utils.vis_utils import plot_model
from keras.layers import Dropout, Dense, LSTM

# Import cufflinks for visualization
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] =(20,10)
plt.style.use('fivethirtyeight')

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Feature selection
import seaborn as sns
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
# Feature importance
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt

# Models
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
#from skpp import ProjectionPursuitRegressor
#from skpp import ProjectionPursuitClassifier
# # ANNs
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import load_model, Model

# Model Metrics
from sklearn.metrics import accuracy_score
# ROC Curve
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

# Resample
# Block Bootstrap
#from arch.bootstrap import StationaryBootstrap
# Time Series cross-validation
#from sklearn.model_selection import TimeSeriesSplit

# Class encoder
#from sklearn.preprocessing import LabelEncoder 

In [2]:
seed_value= 4

In [3]:
df = pd.read_csv('DATA_FRAME_CURATED.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'DATA_FRAME_CURATED.csv'

In [ ]:
df.head(3)

In [ ]:
df.tail(3)

In [ ]:
df = df.dropna()

In [ ]:
df.head(3)

In [ ]:
df.tail(3)

In [ ]:
df.index = df["Date"]

In [ ]:
df.head(3)

In [ ]:
df.tail(3)

In [ ]:
df.drop(["Date"],1,inplace=True)

In [ ]:
df

In [ ]:
df.shape

In [ ]:
list(df.keys())

In [ ]:
X=df.copy()

In [ ]:
X.head()

In [ ]:
Y = pd.DataFrame({'sign': np.sign(X['Y_next_returns'])})

In [ ]:
X.drop("Y_next_returns",1,inplace=True)

In [ ]:
# 70% train/test split
split = int(len(X) * 0.7)

# We transform Y to a binary classification problem's values (0 y 1)
from sklearn.preprocessing import LabelEncoder 
encoder = LabelEncoder() 
encoder.fit(Y) 
encoded_Y = encoder.transform(Y) 
# We transform encoded_Y into a dataframe in order to use .iloc and being able to make the train/test split
Y = pd.DataFrame(encoded_Y, columns = ['Column_A'],index=Y.index) 

# Create train data set
# We have to twist the dataset
X_train, y_train = X[:split], Y[:split]
# Test data after train split
# PARA EVITAR CORRELACION ENTRE LAS SERIES PODEMOS TOMAR [SPLIT+3:] ie QUE PREDIGA 3 DIAS ADELANTE DEL TRAIN SET
X_test, y_test = X[split:], Y[split:]


# Scale the features MinMax for training and test datasets
scaler = MinMaxScaler()
scaled_X_train = scaler.fit_transform(X_train)
scaled_X_test = scaler.transform(X_test)
# We change from array to dataframe
X_train=pd.DataFrame(scaled_X_train,columns=X.columns,index=X.iloc[:split].index)
y_train=pd.DataFrame(y_train,columns=Y.columns,index=Y[:split].index)
X_test=pd.DataFrame(scaled_X_test,columns=X.columns,index=X[split:].index)
y_test=pd.DataFrame(y_test,columns=Y.columns,index=Y[split:].index)

In [ ]:
runtimes=[]

# 1. Naive Bayes

In [ ]:
mnb = MultinomialNB()

start_mnb = time.time()
mnb.fit(X_train, y_train)
end_mnb = time.time()
mnb_runtime=end_mnb - start_mnb

print('Runtime:',mnb_runtime)
print("score on test: %.3f"  %mnb.score(X_test, y_test))
print("score on train: %.3f" %mnb.score(X_train, y_train))

In [ ]:
runtimes.append(mnb_runtime)

In [ ]:
runtimes

In [ ]:
coef=list(mnb.fit(X_train, y_train).coef_[0])
max_coef_index=list(np.argsort(coef)[::-1])
for i in max_coef_index:
    print(list(X_train.columns)[i]+' --> %.3f' %coef[i])

# 2. Logistic Regression

In [ ]:
lr=LogisticRegression(max_iter=1000)

start_lr = time.time()
lr.fit(X_train, y_train)
end_lr = time.time()
lr_runtime=end_lr - start_lr

print('Runtime:',lr_runtime)
print("score on test: %.3f" % lr.score(X_test, y_test))
print("score on train: %.3f" % lr.score(X_train, y_train))

In [ ]:
runtimes.append(lr_runtime)

In [ ]:
coef=list(lr.fit(X_train, y_train).coef_[0])
max_coef_index=list(np.argsort(coef)[::-1])
for i in max_coef_index:
    print(list(X_train.columns)[i]+' --> %.3f' %coef[i])

# 3. K-nearest Neighbours

In [ ]:
knn = KNeighborsClassifier(algorithm = 'brute', n_jobs=-1)

start_knn = time.time()
knn.fit(X_train, y_train)
end_knn = time.time()
knn_runtime=end_knn - start_knn

print('Runtime:',knn_runtime)
print("score on test: %.3f" % knn.score(X_test, y_test))
print("score on train: %.3f" % knn.score(X_train, y_train))

In [ ]:
runtimes.append(knn_runtime)

# 4. Support Vector Machine (SVM)

In [ ]:
svm=LinearSVC(C=100)

start_svm = time.time()
svm.fit(X_train, y_train)
end_svm = time.time()
svm_runtime=end_svm - start_svm

print('Runtime:',svm_runtime)
print("score on test: %.3f" % svm.score(X_test, y_test))
print("score on train: %.3f" % svm.score(X_train, y_train))

In [ ]:
runtimes.append(svm_runtime)

In [ ]:
coef=list(svm.fit(X_train, y_train).coef_[0])
max_coef_index=list(np.argsort(coef)[::-1])
for i in max_coef_index:
    print(list(X_train.columns)[i]+' --> %.3f' %coef[i])

# 5. Decision Tree Classifier

In [ ]:
clf = DecisionTreeClassifier(max_depth=3)

start_clf = time.time()
clf.fit(X_train, y_train)
end_clf = time.time()
clf_runtime=end_clf - start_clf

print('Runtime:',clf_runtime)
print("score on test: %.3f"  % clf.score(X_test, y_test))
print("score on train: %.3f" % clf.score(X_train, y_train))

In [ ]:
runtimes.append(clf_runtime)

In [ ]:
coef=list(clf.fit(X_train, y_train).feature_importances_)
max_coef_index=list(np.argsort(coef)[::-1])
for i in max_coef_index:
    print(list(X_train.columns)[i]+' --> %.3f' %coef[i])

# 6. Bagging Decision Tree (Ensemble Learning I)

In [ ]:
# max_samples: maximum size 0.5=50% of each sample taken from the full dataset
# max_features: maximum of features 1=100% taken here all 10K 
# n_estimators: number of decision trees 
bg=BaggingClassifier(DecisionTreeClassifier(),max_samples=0.5,max_features=1.0,n_estimators=10)

start_bg = time.time()
bg.fit(X_train, y_train)
end_bg = time.time()
bg_runtime=end_bg - start_bg

print('Runtime:',bg_runtime)
print("score on test: %.3f" % bg.score(X_test, y_test))
print("score on train: %.3f" % bg.score(X_train, y_train))

In [ ]:
runtimes.append(bg_runtime)

# 7. Boosting Decision Tree (Ensemble Learning II)

In [ ]:
adb = AdaBoostClassifier(DecisionTreeClassifier(min_samples_split=10,max_depth=4),n_estimators=10,learning_rate=0.6)

start_adb = time.time()
adb.fit(X_train, y_train)
end_adb = time.time()
adb_runtime=end_adb - start_adb

print('Runtime:',adb_runtime)
print("score on test: %.3f" % adb.score(X_test, y_test))
print("score on train: %.3f" % adb.score(X_train, y_train))

In [ ]:
coef=list(adb.fit(X_train, y_train).feature_importances_)
max_coef_index=list(np.argsort(coef)[::-1])
for i in max_coef_index:
    print(list(X_train.columns)[i]+' --> %.3f' %coef[i])

# 8. Random Forest (Ensemble Learning III)

In [ ]:
# n_estimators = number of decision trees
rf = RandomForestClassifier(n_estimators=30, max_depth=9)

start_rf = time.time()
rf.fit(X_train, y_train)
end_rf = time.time()
rf_runtime=end_rf - start_rf

print('Runtime:',rf_runtime)
print("score on test: %.3f" % rf.score(X_test, y_test))
print("score on train: %.3f" % rf.score(X_train, y_train))

In [ ]:
runtimes.append(rf_runtime)

In [ ]:
coef=list(rf.fit(X_train, y_train).feature_importances_)
max_coef_index=list(np.argsort(coef)[::-1])
for i in max_coef_index:
    print(list(X_train.columns)[i]+' --> %.3f' %coef[i])

# 11. Redes neuronales y Deep Learning

In [ ]:
# MLP: mirar https://machinelearningmastery.com/binary-classification-tutorial-with-the-keras-deep-learning-library/ + https://scikit-learn.org/stable/modules/neural_networks_supervised.html
# define model

def model_MLP(optimizer,activation,loss,metrics):
    model = Sequential()

    model.add(Dense(64,  activation = activation))

    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

    return model

MLP=model_MLP(optimizer='adam',activation='relu',loss='binary_crossentropy',metrics=['binary_accuracy'])

In [ ]:
start_MLP = time.time()
history_MLP=MLP.fit(X_train, y_train, batch_size=1, epochs=100, verbose=0, validation_split=0.33, shuffle=False)
end_MLP = time.time()
MLP_runtime=end_MLP - start_MLP

print('Runtime:',MLP_runtime)

In [ ]:
runtimes.append(MLP_runtime)

In [ ]:
print(MLP.summary())

In [ ]:
# MLP
fig, axs = plt.subplots(1,2)
fig.suptitle('MLP model training performance', fontsize=20)
axs[0].plot(history_MLP.history['binary_accuracy'])
axs[0].plot(history_MLP.history['val_binary_accuracy'])
axs[0].set_title('Model accuracy', fontsize=15)
axs[0].set_ylabel('binary_accuracy')
axs[0].set_xlabel('epoch')
axs[0].legend(['train', 'validation'], loc='upper left')
axs[1].plot(history_MLP.history['loss'])
axs[1].plot(history_MLP.history['val_loss'])
axs[1].set_title('Model loss', fontsize=15)
axs[1].set_ylabel('loss')
axs[1].set_xlabel('epoch')
axs[1].legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
# MLP
results_MLP=MLP.evaluate(X_train, y_train)
print('MLP: [binary_crossentropy, binary_accuracy] =', results_MLP)

In [ ]:
# MLP
results_MLP=MLP.evaluate(X_test, y_test)
print('MLP: [binary_crossentropy, binary_accuracy] =', results_MLP)

# 12. LSTM

In [ ]:
# Reshaping the Data for LSTM
# reshaping into 3D Array
X_train_LSTM=np.array(scaled_X_train).reshape(len(scaled_X_train),1,len(list(X.keys()))) # len(scaled_train_data) samples; 1 timesteps per sample; 9 features per timestep or 1 samples; len(scaled_train_data) timesteps; 9 features per timestep?
X_test_LSTM=np.array(scaled_X_test).reshape(len(scaled_X_test),1,len(list(X.keys())))

In [ ]:
def model_LSTM_(optimizer,activation,loss,metrics):
    model = Sequential()

    model.add(LSTM(units=256, input_shape = (X_train_LSTM.shape[1],X_train_LSTM.shape[2]), return_sequences=True)) 
    model.add(Dropout(0.4, seed=seed_value))

    model.add(LSTM(units=256, return_sequences=True))
    model.add(Dropout(0.4, seed=seed_value))

    model.add(LSTM(units=64, return_sequences=False))
    model.add(Dropout(0.4, seed=seed_value))

    model.add(Dense(64,  activation = activation))

    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

    return model

# Define the Model
model_LSTM=model_LSTM_(optimizer='adam',activation='relu',loss='binary_crossentropy',metrics=['binary_accuracy'])

In [ ]:
start_LSTM = time.time()
history_LSTM=model_LSTM.fit(X_train_LSTM, y_train, batch_size=len(X_train_LSTM), epochs=200, verbose=0, validation_split=0.33, shuffle=False)
end_LSTM = time.time()
LSTM_runtime=end_LSTM - start_LSTM

print('Runtime:',LSTM_runtime)

In [ ]:
runtimes.append(LSTM_runtime)

In [ ]:
print(model_LSTM.summary())

In [ ]:
# LSTM
fig, axs = plt.subplots(1,2)
fig.suptitle('LSTM model training performance', fontsize=20)
axs[0].plot(history_LSTM.history['binary_accuracy'])
axs[0].plot(history_LSTM.history['val_binary_accuracy'])
axs[0].set_title('Model accuracy', fontsize=15)
axs[0].set_ylabel('binary_accuracy')
axs[0].set_xlabel('epoch')
axs[0].legend(['train', 'validation'], loc='upper left')
axs[1].plot(history_LSTM.history['loss'])
axs[1].plot(history_LSTM.history['val_loss'])
axs[1].set_title('Model loss', fontsize=15)
axs[1].set_ylabel('loss')
axs[1].set_xlabel('epoch')
axs[1].legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
# LSTM
results_LSTM=model_LSTM.evaluate(X_train_LSTM, y_train)
print('LSTM: [binary_crossentropy, binary_accuracy] =', results_LSTM)

In [ ]:
# LSTM
results_LSTM=model_LSTM.evaluate(X_test_LSTM, y_test)
print('LSTM: [binary_crossentropy, binary_accuracy] =', results_LSTM)